In [1]:
!pip install graphviz
!pip install matplotlib
!pip install tensorflow
!pip install keras

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from time import perf_counter 
import os
import keras as keras
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from tensorflow.keras.utils import plot_model

In [12]:
batch_size = 100
img_height = 250
img_width = 250
training_data = tf.keras.preprocessing.image_dataset_from_directory(
    'data/train',
    seed=42,
    image_size= (img_height, img_width),
    batch_size=batch_size,
    color_mode='rgb'
)

Found 791 files belonging to 2 classes.


In [13]:
validation_data =  tf.keras.preprocessing.image_dataset_from_directory(
    'data/val',
    seed=42,
    image_size= (img_height, img_width),
    batch_size=batch_size,
    color_mode='rgb'
)

testing_data = tf.keras.preprocessing.image_dataset_from_directory(
    'data/test',
    seed=42,
    image_size= (img_height, img_width),
    batch_size=batch_size,
    color_mode='rgb'
)

Found 98 files belonging to 2 classes.
Found 100 files belonging to 2 classes.


In [14]:
class_names = training_data.class_names
class_names

['Accident', 'Non Accident']

In [15]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
training_data = training_data.cache().prefetch(buffer_size=AUTOTUNE)
testing_data = testing_data.cache().prefetch(buffer_size=AUTOTUNE)

In [16]:
model = tf.keras.models.Sequential([
  layers.BatchNormalization(),
  layers.Conv2D(16, 3, activation='relu'), #32 64 128 256 512(Dense)
  layers.MaxPooling2D(), 
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, activation='relu'),
  layers.MaxPooling2D(),
  #layers.Conv2D(128, 3, activation='relu'),
  #layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(32, activation='relu'),
  #layers.Dense(512, activation='relu'),
  layers.Dense(len(class_names), activation= 'softmax')
])

#model.compile(optimizer='adam',loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer='adamax',loss= 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [17]:
model.build((None, 250, 250, 3))

In [ ]:
checkpoint = ModelCheckpoint("model_weights.keras", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
history = model.fit(training_data, validation_data=validation_data, epochs = 30, callbacks=callbacks_list)

Epoch 1/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 813ms/step - accuracy: 0.5164 - loss: 2.3079
Epoch 1: val_accuracy improved from -inf to 0.46939, saving model to model_weights.keras
8/8 ━━━━━━━━━━━━━━━━━━━━ 10s 911ms/step - accuracy: 0.5141 - loss: 2.2552 - val_accuracy: 0.4694 - val_loss: 2.3785
Epoch 2/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 964ms/step - accuracy: 0.4801 - loss: 0.7578
Epoch 2: val_accuracy improved from 0.46939 to 0.61224, saving model to model_weights.keras
8/8 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 0.4818 - loss: 0.7543 - val_accuracy: 0.6122 - val_loss: 0.6821
Epoch 3/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 948ms/step - accuracy: 0.6612 - loss: 0.6852
Epoch 3: val_accuracy did not improve from 0.61224
8/8 ━━━━━━━━━━━━━━━━━━━━ 8s 997ms/step - accuracy: 0.6571 - loss: 0.6854 - val_accuracy: 0.5816 - val_loss: 0.6916
Epoch 4/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 942ms/step - accuracy: 0.5666 - loss: 0.6900
Epoch 4: val_accuracy did not improve from 0.61224
8/8 ━━━━━━━━━━━━━━━━━━━━ 8s 992ms/step

In [ ]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
plt.plot(history.history['loss'], label = 'training loss')
plt.plot(history.history['accuracy'], label = 'training accuracy')
plt.grid(True)
plt.legend()

In [ ]:
plt.plot(history.history['val_loss'], label = 'validation loss')
plt.plot(history.history['val_accuracy'], label = 'validation accuracy')
plt.grid(True)
plt.legend()

In [ ]:
AccuracyVector = []
plt.figure(figsize=(30, 30))
for images, labels in testing_data.take(1):
    predictions = model.predict(images)
    predlabel = []
    prdlbl = []
    
    for mem in predictions:
        predlabel.append(class_names[np.argmax(mem)])
        prdlbl.append(np.argmax(mem))
    
    AccuracyVector = np.array(prdlbl) == labels
    for i in range(40):
        ax = plt.subplot(10, 4, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title('Pred: '+ predlabel[i] +' actl:'+class_names[labels[i]] )
        plt.axis('off')
        plt.grid(True)